# Introduction

We're going to run through one full calculation for the band structure of Silicon. We'll start off by optimizing the structure to minimize the forces, then refining the calculation, and finally running a calcluation for the band structure. 

Goals:
* Use Pymatgen to setup and analyze VASP calculations
* Use Custodian to manage a VASP calculation

# Setup

Let's start by setting up some imports for plotting, pretty printing, and file handling.

Then we'll make a temporary directory to work in

In [ ]:
%matplotlib inline
import pprint
import os
import tarfile
import glob
import matplotlib

In [ ]:
os.mkdir("Si_BandStructure")
os.chdir("Si_BandStructure")

# Building a VASP input set

In this section, we'll build a vasp input set by 'hand' using the structures in Pymatgen. All DFT codes have some sort of input deck or set. This tells the code the parameters of the calculations, the structure to work with, and any input data needed for the calculation. For Pymatgen, the VASP interface is the best developed since it is the code being used for the Materials Project.

A VASP input set consists of:
* POSCAR - the file that defines the structure
* POTCAR - the pseudopotentials that are used in the calculation
* INCAR - input commands that determine what kind of calcluation to run and to what tolerances
* KPOINTS - The recipricol space mesh that the calculations are run on

Let's build each of these and examine their structure in python and as actual input files to VASP. First we'll import the silicon structure as a pymatgen structure. 

In [ ]:
from pymatgen.util.testing import PymatgenTest 
struct_si = PymatgenTest.get_structure("Si")
print(struct_si)

Next, we'll build a POSCAR and look at it's structure. Note that there is no need to examine the structure of the POSCAR file itself since we have a full python interface to this input.

In [ ]:
from pymatgen.io.vasp.inputs import Poscar

Pos = Poscar(struct_si)
print(Pos)

Next, we'll build a KPOiNTS file which gives VASP the recipricoal space mesh that the calculation will be performed. There are several ways to do this in pymatgen. The default constructur is highly flexible and can be used to explcitiy specify kpoints, the shift, coord type, weights, etc. This can be complicated so there are automatic generator schemes that are very powerfull.

In [ ]:
from pymatgen.io.vasp.inputs import Kpoints

kp = Kpoints()
kp = kp.automatic_gamma_density(struct_si,600)
print(kp)

Next, we'll build a POTCAR which contains the potentails used in the calculation. These potentials contain most of the element specific physics. Due to the complexity of these physics, there are several different types of potentials and sub-potentials. Understanding the potentials, the physics they describe, and how to use them is key to running DFT calculations in general, but is outside of the scope of this lesson.

The VASP potentials are proprietary and a license is necessary to actually use them. We can't distribute them. Rather we've set up this Docker container with the potentials so you can explore Pymatgen's ability to build POTCARs

In [ ]:
from pymatgen.io.vasp.inputs import Potcar

pots = Potcar(["Si"])
print(pots)

Let's see how difficult it is to change functionals. 

In [ ]:
pots_lda = Potcar(['Si'],functional="LDA")
print(pots)

The final input file is the INCAR. This file describes the calcluations parameter such as type, what to output, whether to continue a calculation, how precise and accurate to run, etc. In VASP, this file is a series of tags and values. In python that same organization can be achieved with a dictionary. The input for an INCAR object is just a dictionary of tags and values.

In [ ]:
from pymatgen.io.vasp.inputs import Incar

inc = Incar({'NELM': 100, # Max electronic self-consistency steps 
             'IBRION': 2, # Relax using conjugate gradient
             'LWAVE': False, # Don't write wave function to WAVECAR
             'ISMEAR': -5, # Tetrahedron method with Blochl corrections for partial occupancies
             'SIGMA': 0.05, # smearing width in eV
             'MAGMOM': [0.6, 0.6], # Initial magnetic moment
             'ENCUT': 520, # Cutoff energy for plane wave basis set
             'ISIF': 3, # Full degree of freedom and calcualte stress tensor
             u'EDIFF': 0.0001, # Allowed error in total energy
             'NSW': 99, # Max ionic steps
             'LREAL': 'Auto', # Automatically determine if projection done in real space or recipricol space
             'PREC': 'Accurate', # Accurate precision mode
             'ALGO': 'Fast', # Fast electronic minimization algorithm
             'ISPIN': 2, # spin polarized calculations
             'LORBIT': 11 # Write Density of States
            })
print(inc)

# Pymatgen Input Sets 

That was fun, but tedious. There is no way to do that manually for thousands, or even tens of thousands of calculations. What we need is a default set of parameters that is adaptable enough to handle most structures we can throw at it for a specific type of calculation. In Pymatgen, this comes in teh form of an input set. So Let's take the last 4 cells of work and reduce it down to 1, and then write the input to files in the current directory

In [ ]:
from pymatgen.io.vasp.sets import MPRelaxSet

vis = MPRelaxSet(struct_si, force_gamma=True)

print(vis.poscar)
print(vis.kpoints)
print(vis.incar)
print(vis.potcar)
vis.write_input(".")

# Pretend to run VASP

It's not feasible for us to run VASP and wait. Thankfully we've provided you with the appropriate outputs so that you can pretend to run VASP. Let's copy those over.

In [ ]:
!cp ../outputs/Si_Opt/* .
!ls

# Parse outputs

Setting up inputs is great. But, we also need to parse that information. VASP generates a lot of data. 

* OUTCAR - the main output file
* CHGCAR - the charge density
* WAVECAR - the wave functions
* CONTCAR - the relaxed structure
* DOSCAR - the density of states
* vasprun.xml - a xml version of much of the output
* and more .....
Just finding the structure of these files is a lot of work. Thankfully, someone has done that work and put it together into nice python objects. Not all of the possible VASP output data is compiled in these objects, but most of if it is.

Let's take a look at the vasprun.xml file in the python object

In [ ]:
from pymatgen.io.vasp.outputs import Vasprun

vrun = Vasprun("vasprun.xml")
vrun.as_dict()

# Running under custodian
What happens if a calculation fails? When we want to run high throughput calculations, we can't babysit each and every instance of VASP. These silicon calculations are very easy to perform, but a good high throughput search will have many errors that can be easily fixed with a slight modification of the input. This requires a Just In Time (JIT) debugging scheme that can watch the output, fix errors, and rerun VASP or any other pacakge. Custodian is designed to just that. 

First let's clean up the current directory:

In [ ]:
# Clean up 
!mkdir opt
!mv * opt
!ls

Now, let's build a static calculation. The optimization calcualtions allows the postiions of hte ions to relax. Once this is done, we can refine the charge density and wave functions to higher precision, which we call a static calculation. To do this, we need to provide it the output from the optimization calculation. The pymatgen input sets know what information is needed for the calcalation and pull them from the appropriate files when told to pull from a previous calculation:

In [ ]:
from pymatgen.io.vasp.sets import MPStaticSet
static_vis = MPStaticSet.from_prev_calc("./opt")
static_vis.write_input(".")
!ls

Custodian is a just in time job management framework. It has been developed to be generic, although the extensive development under the Materials Project umbrella has resulted in a very capable code base for VASP. There are 3 things custodian needs to run a code like VASP:
* Job - runs the application and does necessart setup and book keeping
* Error Handlers - look for specific problems, identifies them, and can fix them if they are fixable
* Validators - validates the output to ensure the job was run correctly 

The custodian.vasp module contains all three. We'll use this framework to build a simple custodian job that runs a dummy vasp command. We'll use the basic error handler which is designed to account for a number of basic errors that vasp can throw. Finally, we'll validate based on the XML output. 

In [ ]:
from custodian import Custodian
from custodian.vasp.handlers import VaspErrorHandler
from custodian.vasp.jobs import VaspJob
from custodian.vasp.validators import VasprunXMLValidator

handlers = [VaspErrorHandler()]
jobs = [VaspJob(vasp_cmd=["echo","hello"])]
validators = [VasprunXMLValidator()]

c = Custodian(handlers,jobs,validators)
c.run()

Since we can't run vasp, we can't see it's JIT error handling, but it's based on the same type of code as validator. It also fixes and reruns vasp as possible. Some % of production queue is fixed by custodian. To illustrate, we have blank number of jobs.

Let's move the output to this directory to simulate a vasp run and see how custodian works when everything seems fine. 

In [ ]:
# Place pre-run files in Si_BandStructure
!cp ../outputs/Si_Static/* .
  
c.run()

# Exercise - NonSCF Calculation

To finish off this lesson, we'll pretend to run a NonSCF calculation. We'll use the output from the previous static calculation, and calcualte along a line to get a bandstructure using a kpoints line density of 20. Let's prep the previous static calculation output:

In [ ]:
!mkdir static
!mv * static

There is a NonSCF inputset in pymatgen that takes a mode argument that can be set to "line" and a kpoints_line_density. Use that to make the apppropriate input set, inspect the kpoints, write the inputset, and inspect the resulting input files.

In [ ]:
from pymatgen.io.vasp.sets import MPNonSCFSet

NonSCF_vis = MPNonSCFSet.from_prev_calc("./static",mode="Line")
print(NonSCF_vis.kpoints)

In [ ]:
NonSCF_vis.write_input('.')
!ls

# Lets Pretend to run vasp

In [ ]:
!cp ../outputs/Si_NonSCF/* .
!ls

# Plot the band structure

The output of a NonSCF calculation along a set of high symmetry k-points is effectively a bandstructure. Getting this info from the vasp output and plotting it is a lot of work. Let's use pymatgen to do that work for us:

In [ ]:
from pymatgen.io.vasp.outputs import Vasprun
from pymatgen.electronic_structure.plotter import BSPlotter

vrun = Vasprun("vasprun.xml")
band_struc = vrun.get_band_structure(kpoints_filename="KPOINTS",line_mode=True)
bs_plotter = BSPlotter(band_struc)
bs_plotter.show()

# Finalize

In [ ]:
os.chdir("..")
!rm -Rf Si_BandStructure